## Whats a Block?

![alt text](fig1.png)


## Whats a Blockchain? 

![alt text](fig2.png)


## Show Demo


## 4 Step Process!
1. Import dependencies
2. Create a block
3. Create a blockchain
4. Print out the blockchain

In [1]:
import peforth

In [2]:
%%f 擴充、修訂一下 peforth 的行為模式，讓它認得 jupyter notebook 下的 globals. Dot . 也改寫了，適合 jupyter notebook 學習環境使用。
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: ... tib. ; // ( tos -- ) A better dot that also prints the entire command line
  /// For experiments that need to show both question and result.
  /// "" . prints the command line only, w/o the TOS.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code # print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


reDef unknown


## Step 1 - Import Dependencies

In [3]:
#generates timestamps
import datetime
#contains hashing algorithms
import hashlib

## Step 2 - Create a Block 

In [5]:
#defining the 'block' data structure
class Block:
    #each block has 7 attributes 
  
    #1 number of the block
    blockNo = 0
    #2 what data is stored in this block?
    data = None
    #3 pointer to the next block
    next = None
    #4 The hash of this block (serves as a unique ID and verifies its integrity)
    #A hash is a function that converts data into a number within a certain range. 
    hash = None
    #5 A nonce is a number only used once  
    nonce = 0
    #6 store the hash (ID) of the previous block in the chain
    previous_hash = 0x0
    #7 timestamp 
    timestamp = datetime.datetime.now()

    #We initialize a block by storing some data in it
    def __init__(self, data):
        self.data = data

    #Function to compute 'hash' of a block
    #a hash acts as both a unique identifier
    #& verifies its integrity
    #if someone changes the hash of a block
    #every block that comes after it is changed 
    #this helps make a blockchain immutable
    def hash(self):
        #SHA-256 is a hashing algorithm that
        # generates an almost-unique 256-bit signature that represents
        # some piece of text
        h = hashlib.sha256()
        #the input to the SHA-256 algorithm
        #will be a concatenated string
        #consisting of 5 block attributes
        #the nonce, data, previous hash, timestamp, & block #
        h.update(
        str(self.nonce).encode('utf-8') +
        str(self.data).encode('utf-8') +
        str(self.previous_hash).encode('utf-8') +
        str(self.timestamp).encode('utf-8') +
        str(self.blockNo).encode('utf-8')
        )
        #returns a hexademical string
        return h.hexdigest()
      
        ## SHOW DEMO 2, change data 

    def __str__(self):
        #print out the value of a block
        return "Block Hash: " + str(self.hash()) + "\nBlockNo: " + str(self.blockNo) + "\nBlock Data: " + str(self.data) + "\nHashes: " + str(self.nonce) + "\n--------------"


## Step 3 - Create a Blockchain

In [8]:
#defining the blockchain datastructure
#consists of 'blocks' linked together
#to form a 'chain'. Thats why its called
#'blockchain'
class Blockchain:
    
    #mining difficulty
    diff = 20
    #2^32. This is the maximum number
    #we can store in a 32-bit number
    maxNonce = 2**32
    #target hash, for mining
    target = 2 ** (256-diff)  # python 真的可以處理這麼大或更大的數字。

    #generates the first block in the blockchain
    #this is called the 'genesis block'
    block = Block("Genesis")
    #sets it as the head of our blockchain
    head = block

    #adds a given block to the chain of blocks
    #the block to be added is the only parameter
    def add(self, block):
        
        #set the hash of a given block
        #as our new block's previous hash
        block.previous_hash = self.block.hash()
        #set the block # of our new block
        #as the given block's # + 1, since
        #its next in the chain
        block.blockNo = self.block.blockNo + 1

        #set the next block equal to 
        #itself. This is the new head 
        #of the blockchain
        self.block.next = block
        self.block = self.block.next

    #Determines whether or not we can add a given block to
    #the blockchain. mine(block) 真正的產出是該 block 的 nonce, nonce 對應 hash 而 hash() 每次都重算。
    def mine(self, block):
        #from 0 to 2^32 
        for n in range(self.maxNonce):
            #is the value of the given block's hash less than our target value?
            if int(block.hash(), 16) <= self.target:
                #if it is,
                #add the block to the chain
                self.add(block)
                print(block)
                %f datetime :> datetime.now() . cr \ 每 mine 一個都要花幾秒到十幾秒的時間
                break
            else:
                block.nonce += 1  # 如果 hash 不出來，就 nonce++ 然後 retry
            # 如果全部失敗怎麼辦？ 
   
    ## Show demo 3 ! Mine a block

In [ ]:
2**2048 # python 真的可以處理這麼大或更大的數字。

## Step 4 - Print the blockchain 

In [9]:
#initialize blockchain
blockchain = Blockchain()
%f datetime :> datetime.now() . cr

#mine 10 blocks
for n in range(10):
    blockchain.mine(Block("Block " + str(n+1)))
    
#print out each block in the blockchain
while blockchain.head != None:
    print(blockchain.head)
    blockchain.head = blockchain.head.next

2019-01-07 15:33:24.069650
Block Hash: f509b3b4799d3d258c687c53503562551af899b5db5a2e6325000851cee99350
BlockNo: 1
Block Data: Block 1
Hashes: 743351
--------------
2019-01-07 15:33:30.251127
Block Hash: c9ac7105078adf69a76f4683cab3ce8090bec0307dfba76d81787df31b60abcc
BlockNo: 2
Block Data: Block 2
Hashes: 799165
--------------
2019-01-07 15:33:36.711447
Block Hash: 5b497f6bf761355b551320254a335f302938edc8bf7d2d05e853d197108a6067
BlockNo: 3
Block Data: Block 3
Hashes: 123142
--------------
2019-01-07 15:33:37.736864
Block Hash: c9bd0327ab904c2541674b7f14a4fb955569a52a17ac9daf60e5934652b94e8d
BlockNo: 4
Block Data: Block 4
Hashes: 126963
--------------
2019-01-07 15:33:38.770275
Block Hash: 410cc6e515362fa5ec01eff129df230519b1c573e91c42a31d7bfb13af15c6e7
BlockNo: 5
Block Data: Block 5
Hashes: 574392
--------------
2019-01-07 15:33:43.550553
Block Hash: 5807dafa07d7e1e3f92dc5c634ee9bd525e86922bf491e6df0a709b817fac40c
BlockNo: 6
Block Data: Block 6
Hashes: 1354916
--------------
2019-01-0

In [4]:
%f datetime :> datetime.now() . cr

2019-01-07 15:29:11.548371


In [ ]:
%f blockchain :> head .
%f blockchain (see)

In [ ]:
%f blockchain .members